# Sentimen Analysis

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
!pip install wordcloud
from wordcloud import WordCloud

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.utils import resample
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Conv1D, Dense, GlobalMaxPooling1D, Dropout
from keras import regularizers

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.5 MB/s eta 0:00:00


In [ ]:
import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stop words) dalam berbagai bahasa.
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Load Data

Load data yang sudah dilakukan preprocessing hingga stemming pada file lainnya yang memerlukan waktu yang lama dan data hasilnya disimpan pada Gdrive.

In [ ]:
# Reload data hasil stemming yang telah disimpan di drive karena proses nya membutuhkan waktu yang lama
df = pd.read_csv("/content/gdrive/MyDrive/Learn/DATA SCIENCE/clean_df_tinder_stamming.csv")
df.head()

,Unnamed: 0.1,Unnamed: 0,content,score,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,0,0,"Nge bug mulu.. Parah!! Mau log in susah, mau b...",1,Nge bug mulu Parah Mau log in susah mau bales ...,nge bug mulu parah mau log in susah mau bales ...,nge bug melulu parah mau log in susah mau bala...,"['nge', 'bug', 'melulu', 'parah', 'mau', 'log'...","['bug', 'parah', 'log', 'susah', 'balas', 'cha...",bug parah log susah balas chat susah wifi berm...
1,1,1,"Aplikasi sangat jelek sekali, sudah topup tpi ...",2,Aplikasi sangat jelek sekali sudah topup tpi t...,aplikasi sangat jelek sekali sudah topup tpi t...,aplikasi sangat jelek sekali sudah top up tapi...,"['aplikasi', 'sangat', 'jelek', 'sekali', 'sud...","['aplikasi', 'jelek', 'top', 'lihat', 'orang',...",aplikasi jelek top lihat orang suka download a...
2,2,2,"Jangan d pake aplikasi beginian,akun gw tiba2 ...",1,Jangan d pake aplikasi beginianakun gw tiba d ...,jangan d pake aplikasi beginianakun gw tiba d ...,jangan di pakai aplikasi beginianakun saya tib...,"['jangan', 'di', 'pakai', 'aplikasi', 'beginia...","['pakai', 'aplikasi', 'beginianakun', 'blokir'...",pakai aplikasi beginianakun blokir pelanggaran...
3,3,3,Tinder tambah menyebalkan ga bisa liat status ...,2,Tinder tambah menyebalkan ga bisa liat status ...,tinder tambah menyebalkan ga bisa liat status ...,tinder tambah menyebalkan ga bisa lihat status...,"['tinder', 'tambah', 'menyebalkan', 'ga', 'bis...","['tinder', 'menyebalkan', 'lihat', 'status', '...",tinder menyebalkan lihat status menerima chat ...
4,4,4,Di swafoto terus padahl ga aneh aneh sebelum b...,1,Di swafoto terus padahl ga aneh aneh sebelum b...,di swafoto terus padahl ga aneh aneh sebelum b...,di swafoto terus padahal ga aneh aneh sebelum ...,"['di', 'swafoto', 'terus', 'padahal', 'ga', 'a...","['swafoto', 'aneh', 'aneh', 'bayar', 'swafoto'...",swafoto aneh aneh bayar swafoto bayar swafoto ...


## Preprocessing

In [ ]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # remove mentions
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # remove hashtag
    text = re.sub(r'RT[\s]', '', text) # remove RT
    text = re.sub(r"http\S+", '', text) # remove link
    text = re.sub(r'[0-9]+', '', text) # remove numbers
    text = re.sub(r'[^\w\s]', '', text) # remove numbers


    text = text.replace('\n', ' ') # replace new line into space
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove all punctuations
    text = text.strip(' ') # remove characters space from both left and right text
    return text

def casefoldingText(text): # Converting all the characters in a text into lower case
    text = text.lower()
    return text

def tokenizingText(text): # Tokenizing or splitting a string, text into a list of tokens
    text = word_tokenize(text)
    return text

def filteringText(text): # Remove stopwors in a text
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(["nge", "niih", "ny", "ye", "ko", "mulu", "kali", "yah", "melulu", "sie", "wah", "waah", "nii", "wkwkwk", "gitu", "iya", "yaa", "gak", "nya", "heh", "eh", "engga", "enggak",
                          "enggaa", "deh", "doang", "dong", "lah", "wah", "nya", "lo", "na", "sih", "ku", "di", "ga", "ya", "gaa", "loh", "kah", "woi", "woii", "woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words
    # Membuat objek stemmer
    factory = StopWordRemoverFactory()
    stemmer = factory.create_stop_word_remover()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in text]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Convert list of words into sentence
    sentence = ' '.join(word for word in list_words)
    return sentence

In [ ]:
# Mengubah type feature 'text_akhir' menjadi string
df['text_akhir'] = df['text_akhir'].astype(str)
# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_stemming'
df['text_stemming'] = df['text_akhir'].apply(tokenizingText)

## Pelabelan

In [ ]:
import csv
import requests
from io import StringIO

# Loads positive lexicon data from GitHub
# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Loads negative lexicon data from GitHub
# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya ke dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")


In [ ]:
# Function to determine sentiment polarity of tweets
# Fungsi untuk menentukan polaritas sentimen dari tweet

def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if (score > 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar dari 0, maka polaritas adalah positif
    elif (score < 0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif
    else:
        polarity = 'netral'
        # Jika skor sentimen sama dengan 0, maka polaritas adalah netral

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [ ]:
# Mengecek jumlah data berdasarkan label
results = df['text_stemming'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
df['polarity_score'] = results[0]
df['polarity'] = results[1]
print(df['polarity'].value_counts())

polarity
negative    6875
positive    2759
netral      1366
Name: count, dtype: int64


In [ ]:
# Mengatasi data imbalance
df_positif = df[(df['polarity']=='positive')]
df_negatif = df[(df['polarity']=='negative')]
df_neutral = df[(df['polarity']=='netral')]

df_positif_upsampled = resample(df_positif,
                                replace=True,
                                n_samples=6875,
                                random_state=45)

df_netral_upsampled = resample(df_neutral,
                                replace=True,
                                n_samples=6875,
                                random_state=45)

df = pd.concat([df_positif_upsampled, df_netral_upsampled, df_negatif])

In [ ]:
df['polarity'].value_counts()

,count
polarity,
positive,6875
netral,6875
negative,6875


# Random Forest & Tf-Idf

In [ ]:
# Menggunakan tf-idf sebagai feature extraction
tfidf = tfidfVectorizer().fit_transform(df['text_akhir'])

In [ ]:
# Membagi data training set dengan testing set
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(tfidf, df['polarity'], test_size=0.3, random_state=42)

In [ ]:
# Membangun model Random Forest
model_rf = RandomForestClassifier()
model_rf.fit(X_train_rf, y_train_rf)

RandomForestClassifier()

In [ ]:
# Menghitung akurasi data testing
y_pred_rf = model_rf.predict(X_test_rf)

rf_accuracy = accuracy_score(y_pred_rf, y_test_rf)
print(f'Accuracy Testing Data | Random Forest {rf_accuracy}')

Accuracy Testing Data | Random Forest 0.9455397543632837


In [ ]:
cf_rf = classification_report(y_pred_rf, y_test_rf)
print(cf_rf)

              precision    recall  f1-score   support

           0       0.90      0.96      0.93      1943
           1       0.99      0.94      0.97      2163
           2       0.95      0.94      0.95      2082

    accuracy                           0.95      6188
   macro avg       0.95      0.95      0.95      6188
weighted avg       0.95      0.95      0.95      6188



# CNN & FastText

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.vec.gz
!gunzip cc.id.300.vec.gz

--2024-11-23 09:32:32--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.244.202.62, 18.244.202.25, 18.244.202.73, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.244.202.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1227018698 (1.1G) [binary/octet-stream]
Saving to: ‘cc.id.300.vec.gz’

cc.id.300.vec.gz    100%[===================>]   1.14G   237MB/s    in 5.1s    

2024-11-23 09:32:37 (230 MB/s) - ‘cc.id.300.vec.gz’ saved [1227018698/1227018698]



In [ ]:
# Tokenisasi kata
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text_akhir'])
vocab_size = len(tokenizer.word_index) + 1

# Konversi teks menjadi urutan token
sequences = tokenizer.texts_to_sequences(df['text_akhir'])

# Padding urutan agar semua memiliki panjang yang sama
maxlen = 100  # Panjang maksimal teks
X = pad_sequences(sequences, padding='post', maxlen=maxlen)

In [ ]:
# Fungsi Callback untuk mengehentikan training saat acc sudah mencapai target
class myCallback(Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('accuracy')>0.9 and logs.get('val_accuracy')>0.85 and logs.get('val_loss') < 0.2 and logs.get('loss') < 0.2:
      print("\nAkurasi telah mencapai target!")
      self.model.stop_training = True
callbacks = myCallback()

In [ ]:
# Memuat pre-trained FastText embeddings
embedding_dim = 300
embeddings_index = {}
fasttext_path = 'cc.id.300.vec'

with open(fasttext_path, encoding='utf-8') as f:
    for line in f:
        values = line.rstrip().split(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [ ]:
# Membuat matriks embedding
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
# Mengubah label berbentuk integer
df['polarity'] = LabelEncoder().fit_transform(df['polarity'])

In [ ]:
# Membagi data menjadi data traingin dan data testing
X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(X, df['polarity'], test_size=0.3, random_state=42)

In [ ]:
# Model CNN dengan pre-trained embeddings
model_cnn = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen, weights=[embedding_matrix], trainable=True),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    Dropout(0.2),
    Conv1D(filters=32, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Kompilasi model
model_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Melatih model CNN
model_cnn.fit(X_train_cnn, y_train_cnn, epochs=10, validation_split=0.2, callbacks=callbacks)

Epoch 1/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 90s 209ms/step - accuracy: 0.5298 - loss: 0.9508 - val_accuracy: 0.8321 - val_loss: 0.4217
Epoch 2/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 136s 194ms/step - accuracy: 0.8763 - loss: 0.3394 - val_accuracy: 0.8885 - val_loss: 0.2712
Epoch 3/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 83s 196ms/step - accuracy: 0.9400 - loss: 0.1793 - val_accuracy: 0.9348 - val_loss: 0.2060
Epoch 4/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 88s 212ms/step - accuracy: 0.9621 - loss: 0.1160 - val_accuracy: 0.9252 - val_loss: 0.2875
Epoch 5/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 160s 256ms/step - accuracy: 0.9691 - loss: 0.0926 - val_accuracy: 0.9373 - val_loss: 0.2164
Epoch 6/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 128s 223ms/step - accuracy: 0.9787 - loss: 0.0642 - val_accuracy: 0.9406 - val_loss: 0.2603
Epoch 7/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 155s 255ms/step - accuracy: 0.9857 - loss: 0.0480 - val_accuracy: 0.9479 - val_loss: 0.3015
Epoch 8/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 125s 214ms/step - accuracy: 0.9835 - lo

In [ ]:
loss, accuracy = model_cnn.evaluate(X_test_cnn, y_test_cnn)

print(f'Accuracy data testing model CNN - FastText: {accuracy}')
print(f'Loss data testing model CNN - FastText: {loss}')

129/129 ━━━━━━━━━━━━━━━━━━━━ 11s 86ms/step - accuracy: 0.9383 - loss: 0.3968
Accuracy data testing model CNN - FastText: 0.939393937587738
Loss data testing model CNN - FastText: 0.3705722987651825


In [ ]:
# Prediksi pada data testing
y_pred_cnn = model_cnn.predict(X_test_cnn)
y_pred_cnn_classes = y_pred_cnn.argmax(axis=1)

# Hitung akurasi model cnn pada data testing
cnn_accuracy = accuracy_score(y_pred_cnn_classes, y_test_cnn)

print(f"Test Accuracy: {cnn_accuracy:.4f}")

129/129 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step
Test Accuracy: 0.9394


In [ ]:
print(classification_report(y_test_cnn, y_pred_cnn_classes, target_names=['Negative', 'Neutral', 'Positive']))

              precision    recall  f1-score   support

    Negative       0.97      0.87      0.92      1371
     Neutral       0.93      0.98      0.95      1371
    Positive       0.93      0.96      0.95      1383

    accuracy                           0.94      4125
   macro avg       0.94      0.94      0.94      4125
weighted avg       0.94      0.94      0.94      4125



In [ ]:
cm = confusion_matrix(y_test_cnn, y_pred_cnn_classes)
cm

array([[1199,   80,   92],
       [  15, 1342,   14],
       [  25,   24, 1334]])

# LSTM & FastText

In [ ]:
df['polarity'].shape

(20625,)

In [ ]:
df['polarity'].dtype

dtype('int64')

In [ ]:
df['polarity'].value_counts()

,count
polarity,
2,6875
1,6875
0,6875


In [ ]:
# Mengubah label integer menjadi bentuk one-hot-encoding
y_polarity_hot_encode = to_categorical(df['polarity'], num_classes=3)

In [ ]:
# Membagi data menjadi data training dan data testing
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X, y_polarity_hot_encode, test_size = 0.2, random_state=42)

In [ ]:
# Model LSTM dengan pre-trained embeddings
model_lstm = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], trainable=True),
    LSTM(256, return_sequences = True),
    Dropout(0.2),
    LSTM(128, return_sequences = True),
    GlobalMaxPooling1D(),  # Reduces (None, 100, 128) to (None, 128)
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dense(3, activation=' softmax')
])

model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Melatih model LSTM
model_lstm.fit(X_train_lstm, y_train_lstm, epochs=10, validation_split=0.2, callbacks=callbacks)

Epoch 1/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 505s 1s/step - accuracy: 0.5881 - loss: 0.8618 - val_accuracy: 0.8603 - val_loss: 0.3479
Epoch 2/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 499s 1s/step - accuracy: 0.9112 - loss: 0.2434 - val_accuracy: 0.9227 - val_loss: 0.2170
Epoch 3/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 474s 1s/step - accuracy: 0.9593 - loss: 0.1324 - val_accuracy: 0.9348 - val_loss: 0.2040
Epoch 4/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9755 - loss: 0.0765
Akurasi telah mencapai target!
413/413 ━━━━━━━━━━━━━━━━━━━━ 510s 1s/step - accuracy: 0.9755 - loss: 0.0765 - val_accuracy: 0.9445 - val_loss: 0.1888


In [ ]:
loss, accuracy = model_lstm.evaluate(X_test_lstm, y_test_lstm)

print(f'Accuracy data testing model LSTM - FastText: {accuracy}')
print(f'Loss data testing model LSTM - FastText: {loss}')

129/129 ━━━━━━━━━━━━━━━━━━━━ 33s 256ms/step - accuracy: 0.9282 - loss: 0.2518
Accuracy data testing model LSTM - FastText: 0.936242401599884
Loss data testing model LSTM - FastText: 0.23328977823257446


In [ ]:
# Prediksi pada data testing
y_pred_lstm = model_lstm.predict(X_test_lstm)
y_pred_lstm_classes = y_pred.argmax(axis=1)  # Ambil indeks kelas dengan probabilitas tertinggi
y_test_lstm_classes = y_test_lstm.argmax(axis=1)  # Jika y_test dalam bentuk one-hot, konversi juga

# Hitung akurasi model lstm pada data testing
accuracy = accuracy_score(y_test_classes, y_pred_classes)
print(f"Test Accuracy: {accuracy:.4f}")

129/129 ━━━━━━━━━━━━━━━━━━━━ 37s 281ms/step
Test Accuracy: 0.9362


In [ ]:
print(classification_report(y_test_classes, y_pred_classes, target_names=['Negative', 'Neutral', 'Positive']))

              precision    recall  f1-score   support

    Negative       0.93      0.93      0.93      1371
     Neutral       0.93      0.93      0.93      1371
    Positive       0.96      0.95      0.95      1383

    accuracy                           0.94      4125
   macro avg       0.94      0.94      0.94      4125
weighted avg       0.94      0.94      0.94      4125



In [ ]:
cm2 = confusion_matrix(y_test_classes, y_pred_classes)
cm

array([[1199,   80,   92],
       [  15, 1342,   14],
       [  25,   24, 1334]])